In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchtext.data as data
import torchtext.datasets as datasets
import pickle
#print (torch.__version__)

0.3.1.post2


In [2]:
class RNN_Text(nn.Module):
    
    def __init__(self, embed_num, class_num):
        super(RNN_Text, self).__init__()
        V = embed_num
        C = class_num
        H = 256
        
        self.embed = nn.Embedding(V, 100)
        self.rnn = nn.LSTM(100,H, bidirectional = True)
        self.out = nn.Linear(H*2, C)
        
    def forward(self, x):
        x = self.embed(x)  # (N, W, D)
        ##x = x.unsqueeze(1)  # (N, Ci, W, D)
        x,(_,__) = self.rnn( x, ( self.h, self.c ) )
        
        logit = self.out(x[-1])
        return logit
    def inithidden(self,b):
        self.h = Variable(torch.randn(2, b, 256))
        self.c = Variable(torch.randn(2, b, 256))
        

In [3]:
class mydataset(data.Dataset):
    @staticmethod
    def sort_key(ex):
        return len(ex.text)
    def __init__(self, text_field, label_field, path=None, examples=None, **kwargs):
        fields = [('text', text_field), ('label', label_field)]
        if examples is None:
            path = self.dirname if path is None else path
            examples = []
            for i,line in enumerate(open(path,'r',encoding='utf-8')):
                if i==0:
                    continue
                line = line.strip().split('\t')
                txt = line[1].split(' ')
                #txt= [ d.split('/')[0] for d in line[1].split(' ') ]
                examples += [ data.Example.fromlist( [txt, line[2]],fields ) ]
        super(mydataset, self).__init__(examples, fields, **kwargs)

In [4]:
text_field = data.Field(fix_length=20)
#text_field = data.Field()
label_field = data.Field(sequential=False, batch_first = True, unk_token = None)

train_data = mydataset(text_field,label_field,path='../nsm/small_ratings_train_tok.txt')

test_data = mydataset(text_field,label_field,path='../nsm/small_ratings_test_tok.txt')

text_field.build_vocab(train_data)
label_field.build_vocab(train_data)

train_iter, test_iter = data.Iterator.splits(
                            (train_data, test_data), 
                            batch_sizes=(100, 1), repeat=False)#, device = -1)
len(text_field.vocab)

21893

In [5]:
rnn = RNN_Text(len(text_field.vocab),2)
optimizer = torch.optim.Adam(rnn.parameters())
rnn.train()


RNN_Text(
  (embed): Embedding(21893, 100)
  (rnn): LSTM(100, 256, bidirectional=True)
  (out): Linear(in_features=512, out_features=2, bias=True)
)

In [6]:

for epoch in range(15):
    z=0
    totalloss = 0
    for batch in train_iter:
        optimizer.zero_grad()
        
        txt = batch.text
        label = batch.label
        #print (txt.size())
        rnn.inithidden(txt.size(1))
        
        pred = rnn(txt)
        #print(pred.size(), label.size())
        #print(label)
        loss = F.cross_entropy(pred, label)
        totalloss += loss.data[0]
        
        loss.backward()
        optimizer.step()
        #print(data,label)
        
    print(epoch,'epoch')  
    print(totalloss)
    
torch.save(rnn,'model/rnn_model.pt')

0 epoch
69.83460813760757
1 epoch
66.15904760360718
2 epoch
54.62819701433182
3 epoch
43.72859424352646
4 epoch
34.153479278087616
5 epoch
26.51490344852209
6 epoch
19.30791373550892
7 epoch
14.81022097915411
8 epoch
10.427546622231603
9 epoch
8.190892105922103
10 epoch
6.217248036991805
11 epoch
4.998983516357839
12 epoch
3.911643920233473
13 epoch
3.387961565516889
14 epoch
3.3510223287157714


C:\Anaconda3\lib\site-packages\torch\serialization.py:159: UserWarning: Couldn't retrieve source code for container of type RNN_Text. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [7]:
correct = 0
incorrect = 0
rnn.eval()
for batch in test_iter:
    txt = batch.text
    label = batch.label
    
    rnn.inithidden(txt.size(1))
    
    pred = rnn(txt)
    _,ans = torch.max(pred,dim=1)
    
    if ans.data[0] == label.data[0]:
        correct += 1    
    else:
        incorrect += 1
    
print ('correct : ', correct)
print ('incorrect : ', incorrect)
print ()


correct :  77
incorrect :  23

